### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [ ]:
!pip install torchmetrics
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.utils.data as data_utils
import torch.nn as nn
import torch.optim as optim
from torchmetrics.regression import MeanSquaredError
from torch.utils.data import Dataset, DataLoader, TensorDataset
import tqdm
import torch.nn.functional as F

In [ ]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

In [ ]:
df = pd.read_csv("data.txt.zip", header=None)
df.head(10)

In [ ]:
df.info

In [31]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [32]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

#обучим
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

#считаем качество
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

#наилучшее константное предсказание с точки зрения MSE — среднее арифметическое меток по trainу
constant_pred = [y_train.mean()] * len(y_test)
rmse_constant = mean_squared_error(y_test, constant_pred, squared=False)
print(f'RMSE constant: {rmse_constant}')

#при константном прогнозе хуже

RMSE: 9.510160711373397
RMSE constant: 10.85246390513634


## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [33]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [34]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [35]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def create_data_loader(X_train, y_train, X_test, y_test):
    train_tensor = torch.utils.data.TensorDataset(torch.tensor(X_train.astype(np.float32)), torch.tensor(y_train))
    train_loader = torch.utils.data.DataLoader(dataset=train_tensor, batch_size=BATCH_SIZE, shuffle=True)
    test_tensor = torch.utils.data.TensorDataset(torch.tensor(X_test.astype(np.float32)), torch.tensor(y_test))
    test_loader = torch.utils.data.DataLoader(dataset=test_tensor, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader
#модель 1
class TorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=INPUT_SIZE, out_features=HIDDEN_SIZE1)
        self.fc2 = nn.Linear(in_features=HIDDEN_SIZE1, out_features=HIDDEN_SIZE2)
        self.out = nn.Linear(in_features=HIDDEN_SIZE2, out_features=OUTPUT_SIZE)
    def forward(self, x):
        x = nn.LeakyReLU()(self.fc1(x))
        x = nn.LeakyReLU()(self.fc2(x))
        x = self.out(x)
        return x

# модель 2
def build_model():
    model = nn.Sequential(

        nn.Linear(in_features=INPUT_SIZE, out_features=HIDDEN_SIZE1),
        nn.ReLU(),

        nn.Linear(in_features=HIDDEN_SIZE1, out_features=HIDDEN_SIZE2),
        nn.ReLU(),

        nn.Linear(in_features=HIDDEN_SIZE2, out_features=OUTPUT_SIZE),
    )

    return model

# разница моделей только в функциях активаций

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [44]:
# определяем параметры, model, optimizer, criterion, mse, train_loader, test_loader
INPUT_SIZE = 90 # количество признаков
HIDDEN_SIZE1 = 50 # просто так
HIDDEN_SIZE2 = 50
OUTPUT_SIZE = 1 # ну регрессия..
LEARNING_RATE = 0.001 # просто так
EPOCHS = 100
BATCH_SIZE = 128

# model = TorchModel()
model = build_model()
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, nesterov=True)
criterion = nn.MSELoss()
train_loader, test_loader = create_data_loader(X_train, y_train, X_test, y_test)

In [48]:
def train_eval(model, optimizer, criterion, train_loader, test_loader):

    for epoch in range(EPOCHS):
        # тренировка
        model.train()
        for x_train, y_train in train_loader:
            y_pred = model(x_train).squeeze()
            loss = torch.sqrt(criterion(y_pred, y_train.float()))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # валидация
        model.eval()
        val_loss = []
        if epoch % 2 == 0:
            with torch.no_grad():
                for x_test, y_test in test_loader:
                    y_pred = model(x_test).squeeze()
                    loss = torch.sqrt(criterion(y_pred, y_test))
                    val_loss.append(loss.numpy())
            # печатаем метрики
            print(f"Epoch: {epoch}, loss: {np.mean(val_loss)}")

train_eval(model, optimizer, criterion, train_loader, test_loader)

Epoch: 0, loss: 86.82891845703125
Epoch: 2, loss: 91.89496612548828
Epoch: 4, loss: 82.7640609741211
Epoch: 6, loss: 50.09684371948242
Epoch: 8, loss: 69.33561706542969
Epoch: 10, loss: 66.44021606445312
Epoch: 12, loss: 9.089938163757324
Epoch: 14, loss: 8.318602561950684
Epoch: 16, loss: 8.457962989807129
Epoch: 18, loss: 8.234057426452637
Epoch: 20, loss: 8.228263854980469
Epoch: 22, loss: 8.18273639678955
Epoch: 24, loss: 8.329277992248535
Epoch: 26, loss: 8.460043907165527
Epoch: 28, loss: 8.27056884765625
Epoch: 30, loss: 8.471128463745117
Epoch: 32, loss: 8.570989608764648
Epoch: 34, loss: 8.34836483001709
Epoch: 36, loss: 8.439382553100586
Epoch: 38, loss: 8.274364471435547
Epoch: 40, loss: 8.638384819030762
Epoch: 42, loss: 8.214567184448242
Epoch: 44, loss: 8.294500350952148
Epoch: 46, loss: 8.251840591430664
Epoch: 48, loss: 8.694552421569824
Epoch: 50, loss: 8.293304443359375
Epoch: 52, loss: 9.085431098937988
Epoch: 54, loss: 8.320042610168457
Epoch: 56, loss: 8.3494424819

я не плачу, это просто слезы...

можно остановить на эпохе с 14 по 92, кроме 52 - там все результаты стабильно хорошие, а на 98 предлагаю не смотреть...

## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

ОТЧЕТ

**БЕЙЗЛАЙН**
модель без всяких приколов, без методов регуляризации и нормировки таргета
с 14-й эпохи лосс застревает на 12-ти и ни туда, ни сюда

модель - 3 полносвязных слоя, 2 функции активации(ReLU)

оптимизатор - Адам

метрика - MSELoss, из которого я беру корень, чтобы получить RMSE

Epoch: 0, loss: 139.52838134765625

Epoch: 2, loss: 140.1300048828125

Epoch: 4, loss: 113.22109985351562

Epoch: 6, loss: 108.7522201538086

Epoch: 8, loss: 93.57054138183594

Epoch: 10, loss: 53.852272033691406

Epoch: 12, loss: 10.68622875213623

Epoch: 14, loss: 12.874173164367676

Epoch: 16, loss: 12.602913856506348

Epoch: 18, loss: 12.697125434875488

Epoch: 20, loss: 12.653617858886719

Epoch: 22, loss: 12.87939167022705

и т.д.

**что мы можем сделать дальше:**
1. попробовать разные оптимизаторы (на паре мы разобрали SGD, SGD с Nesterov Momentum, RMSprop, Adam - можно начать с этого)
2. стратегии с постепенным понижением lr
3. методы регуляризации, например, dropout и weight decay
4. нормировка таргета

**начала с нормировки таргета**(скопипастила код из тетрадки для StandardScaler)

что получилось:

Epoch: 0, loss: 21.49457359313965

Epoch: 2, loss: 15.128954887390137

Epoch: 4, loss: 16.115442276000977

Epoch: 6, loss: 22.802265167236328

Epoch: 8, loss: 11.895721435546875

Epoch: 10, loss: 22.3963565826416

Epoch: 12, loss: 12.845952033996582

Epoch: 14, loss: 10.230569839477539

Epoch: 16, loss: 13.71241569519043

Epoch: 18, loss: 13.096126556396484

Epoch: 20, loss: 13.875551223754883

Epoch: 22, loss: 18.023155212402344

Epoch: 24, loss: 12.96262264251709

лосс скачет - такое не нравится,
но как будто нормализовать надо, так что оставлю (хуже не стало)

**попробуем другой оптимизатор**

начнем с SGD:

стало хуже, надежда покидает эту тетрадку

Epoch: 0, loss: 64.96495819091797

Epoch: 2, loss: 38.51618576049805

Epoch: 4, loss: 49.212833404541016

Epoch: 6, loss: 54.370906829833984

Epoch: 8, loss: 43.79151916503906

Epoch: 10, loss: 43.910736083984375

Epoch: 12, loss: 41.21782684326172

Epoch: 14, loss: 45.677703857421875

**SGD с Nesterov Momentum**

пока из всего, что было - **лучший результат**, но это настораживает, модель вроде простая, странно, что так хорошо...


**на 22-й эпохе результат 8.18**.............


параметры, которые показали лучший результат:

```
INPUT_SIZE = 90 # количество признаков
HIDDEN_SIZE = 50 # просто так
HIDDEN_SIZE = 
OUTPUT_SIZE = 1 # ну регрессия..
LEARNING_RATE = 0.001 # просто так
EPOCHS = 100
BATCH_SIZE = 128

model = build_model() # тут все очень обычно, архитектуру не меняла
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, nesterov=True)
criterion = nn.MSELoss()
train_loader, test_loader = create_data_loader(X_train, y_train, X_test, y_test)
```


ИТОГО:
я попробовала 3 разных оптимизатора (ADAM, SGD и SGD с Nesterov Momentum), добавила нормировку таргета и поменяла у модели функцию активации с LeakyRelu на Relu (последнее, думаю, не сильно повлияло), где то между этим меняла HIDDEN_SIZE, но 50 на двух слоях оказалось нормально (в самом начале )

больше всего повлиял выбор оптимизатора

не уверена, что стоит менять HIDDEN_SIZE, можно попробовать стратегию с понижением LEARNING_RATE, но уже и так все хорошо работает!!

копипастила из тетрадок структуру моделей, даталоадер, функцию для test_eval, но (естественно) меняла параметры, функции активации и т.д.


P.S. прогнала всю тетрадку еще раз (на всякий случай) и мне поплохело, когда увидела, что лосс растет, но после 10-й эпохи все стало норм

Epoch: 0, loss: 50.518192291259766

Epoch: 2, loss: 88.33194732666016

Epoch: 4, loss: 96.55126190185547

Epoch: 6, loss: 103.18949127197266

Epoch: 8, loss: 64.23108673095703

Epoch: 10, loss: 74.18192291259766

Epoch: 12, loss: 8.650042533874512

Epoch: 14, loss: 8.221396446228027

Epoch: 16, loss: 8.801286697387695

Epoch: 18, loss: 8.362936973571777

Epoch: 20, loss: 8.606343269348145

Epoch: 22, loss: 8.195330619812012